In [1]:
import tensorly as tl
from tensorly.decomposition import parafac, quantized_parafac
from tensorly.kruskal_tensor import kruskal_to_tensor, KruskalTensor
from tensorly.base import unfold
from tensorly.quantization import quantize_qint

import torch
tl.set_backend('pytorch')

# Example 1. Tensor quantization
Quantization scheme can be either affine or symmetric.

Scale and zero_point values to perform quantization are computed either per channel or per tensor (i.e. we get either vectors or scalars).

Thus, there are 4 types of quantization scheme:

    ``torch.per_tensor_affine``
    ``torch.per_tensor_symmetric``
    ``torch.per_channel_affine``
    ``torch.per_channel_symmetric``

##### Generate a random tensor

In [2]:
t = torch.randn(256, 256, 9)
print('||float_tensor|| = {}'.format(tl.norm(t)))

dtype = torch.qint8

||float_tensor|| = 768.3004150390625


##### Per channel  quantization

In [3]:
for qscheme in [torch.per_channel_affine, torch.per_channel_symmetric]:
    print("\nPer channel quantization, dtype: {}, qscheme: {}".format(dtype, qscheme))
    
    for dim in range(len(t.shape)):
        qt, scale, zero_point = quantize_qint(t,\
                                              dtype,\
                                              qscheme,\
                                              dim = dim,\
                                              return_scale_zeropoint=True)

        print('Per dim {}, ||float_tensor - quant_tensor|| = {}'.format(dim, tl.norm(t - qt)))


Per channel quantization, dtype: torch.qint8, qscheme: torch.per_channel_affine
Per dim 0, ||float_tensor - quant_tensor|| = 7.091872692108154
Per dim 1, ||float_tensor - quant_tensor|| = 7.260591983795166
Per dim 2, ||float_tensor - quant_tensor|| = 7.5831427574157715

Per channel quantization, dtype: torch.qint8, qscheme: torch.per_channel_symmetric
Per dim 0, ||float_tensor - quant_tensor|| = 7.091872692108154
Per dim 1, ||float_tensor - quant_tensor|| = 7.260591983795166
Per dim 2, ||float_tensor - quant_tensor|| = 7.5831427574157715


##### Per tensor quantization

In [4]:
for qscheme in [torch.per_tensor_affine, torch.per_tensor_symmetric]:
    print("\nPer tensor quantization, dtype: {}, qscheme: {}".format(dtype, qscheme))

    
    qt, scale, zero_point = quantize_qint(t,\
                                          dtype,\
                                          qscheme,\
                                          dim = dim,\
                                          return_scale_zeropoint=True)
    print('Per tensor, ||float_tensor - quant_tensor|| = {}'.format(tl.norm(t - qt)))


Per tensor quantization, dtype: torch.qint8, qscheme: torch.per_tensor_affine
Per tensor, ||float_tensor - quant_tensor|| = 8.514548301696777

Per tensor quantization, dtype: torch.qint8, qscheme: torch.per_tensor_symmetric
Per tensor, ||float_tensor - quant_tensor|| = 8.514548301696777


# Example 2. Quantization of a tensor in Kruskal format:
    a) via quantization of the corresponding full tensor
    b) via quantization of decomposition factors.

##### Generate tensor 

In [5]:
rank = 16
shape = (64, 64, 9)

factors = [torch.randn((i, rank)) for i in shape] 
weights = torch.ones(rank)

# tensor in Kruscal format
krt = KruskalTensor((weights, factors))

# corresponding tensor in full format
t = kruskal_to_tensor(krt)

tnorm = tl.norm(t)
print('||float_factors||: {}'.format(tnorm))

||float_factors||: 710.7957763671875


##### Choose quantization scheme

In [6]:
dtype = torch.qint8

## Per tensor quantization
qscheme, dim = torch.per_tensor_affine, None

## Uncomment for per channel quantization
# qscheme, dim = torch.per_channel_affine, 0

##### a) Quantize the full tensor

In [7]:
t_quant = quantize_qint(t, dtype, qscheme, dim = dim)
print('||float_factors - float_factors_quantized|| = {}'.format(tl.norm(t - t_quant)))

||float_factors - float_factors_quantized|| = 9.844650268554688


##### b) Quantize several factors

In [8]:
num_factors = len(factors)
for num_quant_factors in range(1, num_factors + 1):
    
    qfactors = [quantize_qint(factors[i], dtype, qscheme, dim = dim)\
                for i in range(num_quant_factors)\
               ] + [factors[i] for i in range(num_quant_factors, num_factors)]

    qkrt = KruskalTensor((weights, qfactors))
    qt = kruskal_to_tensor(qkrt)
    print('\n[{}/{}] factors are quantized'.format(num_quant_factors, num_factors))
    print('||quant_factors - float_factors|| = {}'.format(tl.norm(qt - t)))
#     print('||quant_factors - float_factors_quantized|| = {}'.format(tl.norm(qt - t_quant)))

#     qt_quant = quantize_qint(qt, dtype, qscheme, dim = dim)
#     print('\nquant_factors_quantized - t_quant_factors: {}'.format(tl.norm(qt_quant - qt)/tnorm))
    
#     print('||quant_factors_quantized - float_factors|| = {}'.format(tl.norm(qt_quant - t)/tnorm))
#     print('||quant_factors_quantized - float_factors_quantized|| = {}'.format(tl.norm(qt_quant - t_quant)/tnorm))



[1/3] factors are quantized
||quant_factors - float_factors|| = 5.352328777313232

[2/3] factors are quantized
||quant_factors - float_factors|| = 7.942391872406006

[3/3] factors are quantized
||quant_factors - float_factors|| = 18.10138702392578


# Example 3. Quantized ALS
Compare standard ALS algorithm for finding  CP decomposition with its quantized version, when at the end of each ALS step approximated factor is quantized.

##### Generate tensor

In [9]:
rank = 8
shape = (128, 128, 9)

factors = [torch.randn((i, rank)) for i in shape] 
weights = torch.ones(rank)

# tensor in Kruscal format
krt = KruskalTensor((weights, factors))

# corresponding tensor in full format
t = kruskal_to_tensor(krt)

tnorm = tl.norm(t)
print('||float_factors||: {}'.format(tnorm))

||float_factors||: 896.6329956054688


##### Find an approximation using ALS

In [22]:
for stop_criterion in ['rec_error_decrease', 'rec_error_deviation']:
    out = parafac(t, rank, n_iter_max=100, init='svd', svd='numpy_svd',\
                normalize_factors=False, orthogonalise=False,\
                tol=1e-8, random_state=None,\
                verbose=0, return_errors=False,\
                non_negative=False, mask=None,
                stop_criterion = stop_criterion)
    print(tl.norm(t - kruskal_to_tensor(out)))  

tensor(0.0007)
tensor(0.0628)


In [10]:
normalize_factors = False
for stop_criterion in ['rec_error_decrease', 'rec_error_deviation']:
    (factors_als, weights_als), _ =  quantized_parafac(t, rank,
                                                       n_iter_max=50000,\
                                                       init='random',\
                                                       tol=1e-8,\
                                                       svd = None,\
                                                       normalize_factors = normalize_factors,\
                                                       stop_criterion = stop_criterion
                                                      )
    print(tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_als, factors_als)))))

tensor(0.0002)
tensor(226.9214)


In [12]:
weights_als

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

##### Find an approximation using quantized ALS

In [15]:
dtype = torch.qint8

## Per tensor quantization
qscheme, dim = torch.per_tensor_affine, None

## Uncomment for per channel quantization
# qscheme, dim = torch.per_channel_affine, 0

In [22]:
normalize_factors = False
(factors_qals, weights_qals), _, scales, zero_points = quantized_parafac(
                                    t, rank, n_iter_max=1001,\
                                    init='random', tol= False, svd = None,\
                                    normalize_factors = normalize_factors,\
                                    qmodes = [0, 1, 2],
                                    warmup_iters = 1,
                                    quantize_every = 1,
                                    qscheme = qscheme, dtype = dtype, dim = dim,
                                    return_scale_zeropoint=True)

In parafac original_tensor_norm = 913.5255737304688
iteration 500, diff_from_norm = 12.325675964355469, rel_rec_error = 0.013492425739131085
iteration 1000, diff_from_norm = 12.325695037841797, rel_rec_error = 0.013492446618115623


In [23]:
tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_qals, factors_qals))))

tensor(12.3257)

In [24]:
weights_qals

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
scales, zero_points

((tensor(0.2144), tensor(0.1242), tensor(0.0004)),
 (tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32)))

In [26]:
factors_qals[0]

tensor([[-12.8624,  -3.6444,   8.5750,  ...,  -2.1437,   2.1437,  -2.5725],
        [ -4.0731,  -2.5725,  -2.7869,  ...,  -8.5750,   2.1437,  12.8624],
        [  0.4287,   0.2144,  11.3618,  ..., -15.8637,   8.3606,  -0.8575],
        ...,
        [ -1.2862,   0.0000,  -6.8600,  ...,  -3.8587,  -8.5750,   9.4325],
        [  8.3606,  -0.6431, -15.0062,  ...,  -6.8600,  -1.5006,  -1.0719],
        [ -3.6444,  -1.0719,   7.2887,  ..., -25.2961,   9.4325,  21.2230]])

In [27]:
factors_qals[0]/scales[0]

tensor([[ -60.0000,  -17.0000,   40.0000,  ...,  -10.0000,   10.0000,
          -12.0000],
        [ -19.0000,  -12.0000,  -13.0000,  ...,  -40.0000,   10.0000,
           60.0000],
        [   2.0000,    1.0000,   53.0000,  ...,  -74.0000,   39.0000,
           -4.0000],
        ...,
        [  -6.0000,    0.0000,  -32.0000,  ...,  -18.0000,  -40.0000,
           44.0000],
        [  39.0000,   -3.0000,  -70.0000,  ...,  -32.0000,   -7.0000,
           -5.0000],
        [ -17.0000,   -5.0000,   34.0000,  ..., -118.0000,   44.0000,
           99.0000]])

In [28]:
factors_qals[1]/scales[1]

tensor([[-18.0000,   2.0000, -21.0000,  ...,  12.0000, -38.0000,   9.0000],
        [ -6.0000, -15.0000,  -2.0000,  ...,  60.0000,  74.0000,   9.0000],
        [ 21.0000, -17.0000, 107.0000,  ..., -32.0000, 113.0000,  63.0000],
        ...,
        [ 40.0000,   0.0000, -16.0000,  ...,  14.0000,   9.0000, -46.0000],
        [ -7.0000, 124.0000,  16.0000,  ...,  31.0000,  26.0000, -30.0000],
        [-14.0000, -51.0000,  88.0000,  ...,   3.0000,  78.0000, -56.0000]])

##### Try to quantize 2 of 3 factors

In [31]:
normalize_factors = False
(factors_qals, weights_qals), _, scales, zero_points = quantized_parafac(
                                    t, rank, n_iter_max=1001,\
                                    init='random', tol= False, svd = None,\
                                    normalize_factors = normalize_factors,\
                                    qmodes = [0, 1],
                                    warmup_iters = 0,
                                    quantize_every = 1,
                                    qscheme = qscheme, dtype = dtype, dim = dim,
                                    return_scale_zeropoint=True)

In parafac original_tensor_norm = 913.5255737304688
iteration 500, diff_from_norm = 21.66213607788086, rel_rec_error = 0.02371267614262999
iteration 1000, diff_from_norm = 21.66213607788086, rel_rec_error = 0.02371267614262999


In [32]:
tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_qals, factors_qals))))

tensor(21.6621)